In [8]:
import pandas as pd
import numpy as np
import data_cleaning
import chicago_api as chi

In [3]:
## Testing restaurant df filtering
chunk_size = 100000
restaurants_df = data_cleaning.filter_business_data("data/yelp_academic_dataset_business.json", chunk_size)

## Testing save and load
data_cleaning.save(restaurants_df, "data/filtered_restaurants.json")
reloaded_restaurants_df = data_cleaning.load("data/filtered_restaurants.json", chunk_size)
reloaded_restaurants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27894 entries, 0 to 27893
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   27894 non-null  object 
 1   name          27894 non-null  object 
 2   address       27894 non-null  object 
 3   city          27894 non-null  object 
 4   state         27894 non-null  object 
 5   postal_code   27894 non-null  object 
 6   latitude      27894 non-null  float64
 7   longitude     27894 non-null  float64
 8   stars         27894 non-null  float64
 9   review_count  27894 non-null  int64  
 10  is_open       27894 non-null  int64  
 11  attributes    27888 non-null  object 
 12  categories    27894 non-null  object 
 13  hours         26538 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 3.0+ MB


In [4]:
## Testing review df filtering
chunk_size = 100000
reviews_df = data_cleaning.filter_review_data("data/yelp_academic_dataset_review.json", restaurants_df, chunk_size)
reviews_df.info()

## Testing save and load
data_cleaning.save(reviews_df, "data/filtered_reviews.json")
reloaded_reviews_df = data_cleaning.load("data/filtered_reviews.json", chunk_size)
reloaded_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4371248 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 333.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4371248 entries, 0 to 4371247
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date        

In [18]:
## Testing users df filtering
chunk_size = 100000
users_df = filter_user_data("data/yelp_academic_dataset_user.json", reviews_df, chunk_size)
users_df.info()

## Testing save and load
save(users_df, "data/filtered_users.json")
reloaded_users_df = load("data/filtered_users.json", chunk_size)
reloaded_users_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1379968 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   user_id             1379968 non-null  object 
 1   name                1379968 non-null  object 
 2   review_count        1379968 non-null  int64  
 3   yelping_since       1379968 non-null  object 
 4   useful              1379968 non-null  int64  
 5   funny               1379968 non-null  int64  
 6   cool                1379968 non-null  int64  
 7   elite               1379968 non-null  object 
 8   friends             1379968 non-null  object 
 9   fans                1379968 non-null  int64  
 10  average_stars       1379968 non-null  float64
 11  compliment_hot      1379968 non-null  int64  
 12  compliment_more     1379968 non-null  int64  
 13  compliment_profile  1379968 non-null  int64  
 14  compliment_cute     1379968 non-null  int64  
 15  compliment_list     

In [9]:
## Pull Chicago API data
#chi_restaurants_df = chi.get_chicago_restaurants_df(0, 10)
#chi_restaurants_df = chi.get_chicago_restaurants_df(10, 60)
#chi_restaurants_df = chi.get_chicago_restaurants_df(60, 110)
#chi_restaurants_df = chi.get_chicago_restaurants_df(110, 160)
#chi_restaurants_df = chi.get_chicago_restaurants_df(160, 210)
chi_restaurants_df = chi.get_chicago_restaurants_df(210, 260)

South Lawndale has 5300 hits


/home/chasty2/Documents/group-project-better-late-than-never/chicago_api.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_restaurants = pd.concat([all_restaurants, restaurants_df], ignore_index=True)


The Loop has 2800 hits
South Shore has 1200 hits
Douglas has 929 hits
Avalon Park has 445 hits
Near North Side has 5500 hits
Mount Greenwood has 364 hits
Near West Side has 5800 hits
West Town has 6300 hits
Norwood Park has 2500 hits
Near West Side has 5800 hits
Uptown has 1400 hits
Irving Park has 6200 hits
Garfield Ridge has 2000 hits
Avondale has 6500 hits
Washington Heights has 441 hits
Washington Park has 548 hits
Armour Square has 1300 hits
Beverly has 443 hits
Chatham has 484 hits
Chatham has 484 hits
Roseland has 1300 hits
Lincoln Park has 6000 hits
West Elsdon has 261 hits
West Englewood has 561 hits
West Garfield Park has 893 hits
Austin has 3800 hits
Humboldt Park has 6800 hits
Lake View has 1800 hits
West Lawn has 578 hits
Near West Side has 5800 hits
Morgan Park has 339 hits
West Pullman has 238 hits
West Ridge has 1100 hits
West Ridge has 1100 hits
West Town has 6300 hits
Woodlawn has 513 hits
West Town has 6300 hits
Forest Glen has 523 hits
Woodlawn has 513 hits
Ashburn 

In [10]:
all_chi_restaurants = data_cleaning.load("data/chicago_restaurants_incomplete", 100_000)

In [11]:
all_chi_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50148 entries, 0 to 50147
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              50148 non-null  object 
 1   alias           50148 non-null  object 
 2   name            50148 non-null  object 
 3   image_url       50148 non-null  object 
 4   is_closed       50148 non-null  bool   
 5   url             50148 non-null  object 
 6   review_count    50148 non-null  int64  
 7   categories      50148 non-null  object 
 8   rating          50148 non-null  float64
 9   coordinates     50148 non-null  object 
 10  transactions    50148 non-null  object 
 11  price           21745 non-null  object 
 12  location        50148 non-null  object 
 13  phone           50148 non-null  object 
 14  display_phone   50148 non-null  object 
 15  distance        50148 non-null  float64
 16  business_hours  50148 non-null  object 
 17  attributes      50148 non-null 

In [12]:
all_chi_restaurants = pd.concat([all_chi_restaurants, chi_restaurants_df], ignore_index=True)
data_cleaning.save(all_chi_restaurants, "data/chicago_restaurants_incomplete")

In [13]:
all_chi_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60466 entries, 0 to 60465
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              60466 non-null  object 
 1   alias           60466 non-null  object 
 2   name            60466 non-null  object 
 3   image_url       60466 non-null  object 
 4   is_closed       60466 non-null  object 
 5   url             60466 non-null  object 
 6   review_count    60466 non-null  object 
 7   categories      60466 non-null  object 
 8   rating          60466 non-null  float64
 9   coordinates     60466 non-null  object 
 10  transactions    60466 non-null  object 
 11  price           26125 non-null  object 
 12  location        60466 non-null  object 
 13  phone           60466 non-null  object 
 14  display_phone   60466 non-null  object 
 15  distance        60466 non-null  float64
 16  business_hours  60466 non-null  object 
 17  attributes      60466 non-null 

In [14]:
# Test flask recommendation

import flask.recommend as rec

cleaned_chi = all_chi_restaurants.dropna()
cleaned_chi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26125 entries, 5 to 60463
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              26125 non-null  object 
 1   alias           26125 non-null  object 
 2   name            26125 non-null  object 
 3   image_url       26125 non-null  object 
 4   is_closed       26125 non-null  object 
 5   url             26125 non-null  object 
 6   review_count    26125 non-null  object 
 7   categories      26125 non-null  object 
 8   rating          26125 non-null  float64
 9   coordinates     26125 non-null  object 
 10  transactions    26125 non-null  object 
 11  price           26125 non-null  object 
 12  location        26125 non-null  object 
 13  phone           26125 non-null  object 
 14  display_phone   26125 non-null  object 
 15  distance        26125 non-null  float64
 16  business_hours  26125 non-null  object 
 17  attributes      26125 non-null  obje

In [6]:
data_cleaning.save(cleaned_chi, "flask/static/test_data.json")

In [5]:
yelp_rec = rec.get_best_restaurants(cleaned_chi, "Seafood", "Albany Park", "$$")
yelp_rec

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,business_hours,attributes,neighborhood
5,AVewNBwjEYHRabcLfoo3xg,wellfleet-occasional-restaurant-chicago,Wellfleet Occasional Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/aLn02l...,False,https://www.yelp.com/biz/wellfleet-occasional-...,7,"[{'alias': 'seafood', 'title': 'Seafood'}]",5.0,"{'latitude': 41.9615402222, 'longitude': -87.7...",[],$$,"{'address1': '4423 N Elston Ave', 'address2': ...",+17732837400,(773) 283-7400,1048.495503,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': Non...",Albany Park
32,nZQtAL-nzsdOMdefBtoroQ,our-mutual-city-coffee-chicago,Our Mutual City Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/vj4tgn...,False,https://www.yelp.com/biz/our-mutual-city-coffe...,21,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",5.0,"{'latitude': 41.9539566736, 'longitude': -87.7...",[],$,"{'address1': '3600 W Irving Park Rd', 'address...",+17736739000,(773) 673-9000,1630.773879,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': 'ht...",Albany Park
31809,AVewNBwjEYHRabcLfoo3xg,wellfleet-occasional-restaurant-chicago,Wellfleet Occasional Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/aLn02l...,False,https://www.yelp.com/biz/wellfleet-occasional-...,7,"[{'alias': 'seafood', 'title': 'Seafood'}]",5.0,"{'latitude': 41.9615402222, 'longitude': -87.7...",[],$$,"{'address1': '4423 N Elston Ave', 'address2': ...",+17732837400,(773) 283-7400,1048.495503,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': Non...",Albany Park
31805,nZQtAL-nzsdOMdefBtoroQ,our-mutual-city-coffee-chicago,Our Mutual City Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/vj4tgn...,False,https://www.yelp.com/biz/our-mutual-city-coffe...,21,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",5.0,"{'latitude': 41.9539566736, 'longitude': -87.7...",[],$,"{'address1': '3600 W Irving Park Rd', 'address...",+17736739000,(773) 673-9000,1630.773879,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': 'ht...",Albany Park
35886,nZQtAL-nzsdOMdefBtoroQ,our-mutual-city-coffee-chicago,Our Mutual City Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/vj4tgn...,False,https://www.yelp.com/biz/our-mutual-city-coffe...,21,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",5.0,"{'latitude': 41.9539566736, 'longitude': -87.7...",[],$,"{'address1': '3600 W Irving Park Rd', 'address...",+17736739000,(773) 673-9000,1630.773879,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': 'ht...",Albany Park
